In [52]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import IFrame
import pandas as pd 
import os
import shutil
import datetime

path = "Resources/"

In [53]:
def save(contents,name):
    f=open(path+name+".js","w+")
    f.write(contents)
    f.close()

    f=open(path+name+".html", "w+")
    f.write('<script src="https://code.highcharts.com/highcharts.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/series-label.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/exporting.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/export-data.js"></script>'+ "\n")
    f.write('<script src="https://code.highcharts.com/modules/sunburst.js"></script>'+ "\n\n")

    f.write("<div id='container'></div>"+ "\n")
    f.write("<script src='"+name+".js'></script>")
    f.close() 
    
def runTCat(data,title,fname):
    contents = Path(path + "templateCat.js").read_text()
    
    contents = contents.replace("#data#",str(data))
    contents = contents.replace("#titulo#",title)
    
    save(contents,fname)

In [54]:
data = pd.read_csv(path + "flags.arff.csv")
data = data.reset_index()
dataset = data.rename(columns={"animate":"class_name"})

In [55]:
def parser(r_pattern):
    pat = r_pattern.split(" ")
    pattern = {"name": pat[0], "operator":pat[1], "value": pat[2]}
    return pattern

def multi_parser(r_patterns):
    patterns = []
    for pattern in r_patterns:
        patterns.append(parser(pattern))
    return patterns

def dividepat(pattern):
    ptrns = pattern.split(" AND ")
    rptr = []
    for ptr in ptrns:
        rptr.append(parser(ptr))
    
    return rptr,ptrns

def query_pattern(options):
    return "{} {} {}".format(options["name"],options["operator"],options["value"])

def n_query_pattern(options):
    op = ""
    if options["operator"] == "==":
        op = "!="
    else:
        op = "=="
    return "{} {} {}".format(options["name"],op,options["value"])

In [56]:
def leafs(dataset,total,parent,options,i,items): 
    ds1 = dataset.query(query_pattern(options[i]))
    ds2 = dataset.query(n_query_pattern(options[i]))

    v1 = len(ds1.index)/total
    v2 = len(ds2.index)/total

    id1 = "{}.{}".format(parent,'0')
    id2 = "{}.{}".format(parent,'1')
    
    cvs = [
            {
          'id': id1,
          'parent': parent,
          'name': "".join(query_pattern(options[i])),
          'value': v1
        },{
          'id': id2,
          'parent': parent,
          'name': "".join(n_query_pattern(options[i])),
          'value': v2
        }
    ]
    
    if items > 1:
        cvs = [*cvs, *leafs(ds1,total,id1,options,i + 1,items - 1), *leafs(ds2,total,id2,options,i + 1,items - 1)]
    return cvs

In [57]:
def ItemCN(patterns, dataset, items):
    options = dividepat(patterns)[0]
        
    classes = dataset["topleft"].values
    classvals = np.unique(classes).tolist()
        
    total = dataset.count()[0]
    csv = [{'id':'0.0','parent':'', 'name': 'All'}] # Root
    csv.append({'id':'1.0', 'parent':'0.0', 'name': "blue" })
    csv.append({'id':'1.1', 'parent':'0.0', 'name': "green" })
    csv.append({'id':'1.2', 'parent':'0.0', 'name': "white" })
    
    csv = csv + leafs(dataset,total,'1.0',options,0,items)
    csv = csv + leafs(dataset,total,'1.1',options,0,items)
    csv = csv + leafs(dataset,total,'1.2',options,0,items)
    
    return csv

In [58]:
def OneItemCM(patterns, dataset):
    csv = ItemCB(patterns, dataset,1)
    runTCat(csv,"OneItemCM","OneItemCM")

OneItemCM("religion == 2 ", dataset)
IFrame(src=path+'OneItemCM.html', width=650, height=650)

In [48]:
def TwoItemCM(patterns, dataset):
    csv = ItemCB(patterns, dataset,2)
    runTCat(csv,"TwoItemCM","TwoItemCM")

TwoItemCM("religion == 2 AND language == 10", dataset)
IFrame(src=path+'TwoItemCM.html', width=650, height=650)

In [50]:
def ThreeItemCM(patterns, dataset):
    csv = ItemCB(patterns, dataset,3)
    runTCat(csv,"ThreeItemCM","ThreeItemCM")

ThreeItemCM("religion == 2 AND language == 10 AND stripes == 3", dataset)
IFrame(src=path+'ThreeItemCM.html', width=650, height=650)

In [51]:
def FourItemCM(patterns, dataset):
    csv = ItemCB(patterns, dataset,4)
    runTCat(csv,"FourItemCM","FourItemCM")

FourItemCM("religion == 2 AND language == 10 AND stripes == 3 AND sunstars == 1", dataset)
IFrame(src=path+'FourItemCM.html', width=650, height=650)